# 네이버 영화평 분석 - TfidfVectorizer

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 앞서 전처리 다 완료한 데이터 불러오기
train_df = pd.read_csv('naver_movie_train.csv',sep='\t')
test_df = pd.read_csv('naver_movie_test.csv',sep='\t')

train_df.head()

,Unnamed: 0,id,document,label
0,0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0
3,3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


### 1. Tokenizer 함수 정의

In [3]:
from konlpy.tag import Okt
okt= Okt()

In [4]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅎㅎ','ㅠㅠ']

In [ ]:
def okt_tokenizer(text):
    tokens = okt.morphs(text,stem=True)
    tokens = [word for word in tokens if not word in stopwords]
    return tokens

### TfidfVectorizer로 변환 

In [14]:
# 경고 제거 
import warnings

warnings.filterwarnings(action='ignore')

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(
    tokenizer=okt_tokenizer, ngram_range=(1,2),
    max_df=0.9, stop_words=stopwords)

In [18]:
%time tvect.fit(train_df.document)

Wall time: 5min 12s


TfidfVectorizer(max_df=0.9, ngram_range=(1, 2),
                stop_words=['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과',
                            '도', '를', '으로', '자', '에', '와', '한', '하다', '을', 'ㅋㅋ',
                            'ㅎㅎ', 'ㅠㅠ'],
                tokenizer=<function okt_tokenizer at 0x000001FE04C0D4C0>)

In [19]:
%time X_train_tv = tvect.transform(train_df.document)

Wall time: 5min 16s


In [20]:
X_test_tv = tvect.transform(test_df.document)

In [21]:
y_train = train_df.label.values
y_test = test_df.label.values

### Naive Bayes 기본 분류기로 학습 / 예측 / 평가

In [22]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [23]:
nb.fit(X_train_tv, y_train)     # 학습 속도가 매우 빠르다 

MultinomialNB()

In [24]:
from sklearn.metrics import accuracy_score
pred = nb.predict(X_test_tv)

0.8608837636493519

In [25]:
accuracy_score(pred,y_test)

0.8608837636493519

실제 테스트

In [26]:
reviews = ['아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요.','메세지와 작위성의 불협화음!!!!']

In [27]:
reviews_tv = tvect.transform(reviews)       # tvect 만들면서 이미 토큰화했기 때문에 정규표현식 필요 없음
pred = nb.predict(reviews_tv)
pred

array([1, 0], dtype=int64)